In [2]:
import pandas as pd
import json

In [4]:
file_path= 'C:/Users/ASUS/Downloads/archive/Sarcasm_Headlines_Dataset_v2.json'
data=[]
with open(file_path, 'r') as f:
    for line in f:
        data.append(json.loads(line))
df = pd.DataFrame(data)
print(df.head())

   is_sarcastic                                           headline  \
0             1  thirtysomething scientists unveil doomsday clo...   
1             0  dem rep. totally nails why congress is falling...   
2             0  eat your veggies: 9 deliciously different recipes   
3             1  inclement weather prevents liar from getting t...   
4             1  mother comes pretty close to using word 'strea...   

                                        article_link  
0  https://www.theonion.com/thirtysomething-scien...  
1  https://www.huffingtonpost.com/entry/donna-edw...  
2  https://www.huffingtonpost.com/entry/eat-your-...  
3  https://local.theonion.com/inclement-weather-p...  
4  https://www.theonion.com/mother-comes-pretty-c...  


In [6]:
!pip install nltk
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [7]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [8]:
stop_words= set(stopwords.words('english'))
lemmatizer=WordNetLemmatizer()
def preprocess(text):
    text= text.lower()
    text=re.sub(r'[^a-z\s]','',text)
    tokens= nltk.word_tokenize(text)
    cleaned= [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return' '.join(cleaned)

In [9]:
df['cleaned_headline']= df['headline'].apply(preprocess)
print(df[['headline', 'cleaned_headline']].head())

                                            headline  \
0  thirtysomething scientists unveil doomsday clo...   
1  dem rep. totally nails why congress is falling...   
2  eat your veggies: 9 deliciously different recipes   
3  inclement weather prevents liar from getting t...   
4  mother comes pretty close to using word 'strea...   

                                    cleaned_headline  
0  thirtysomething scientist unveil doomsday cloc...  
1  dem rep totally nail congress falling short ge...  
2            eat veggie deliciously different recipe  
3       inclement weather prevents liar getting work  
4  mother come pretty close using word streaming ...  


In [10]:
!pip install scikit-learn

In [11]:
import sys
print(sys.executable)

C:\Users\ASUS\AppData\Local\Programs\Python\Python313\python.exe


In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer= TfidfVectorizer(max_features=1000)
X=vectorizer.fit_transform(df['cleaned_headline']).toarray()
y= df['is_sarcastic']

In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test,y_train,y_test=train_test_split(X,y,test_size=0.2, random_state=42)

In [22]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.neighbors import KNeighborsClassifier

In [24]:
nb = MultinomialNB()
nb.fit(X_train, y_train)
y_pred_nb = nb.predict(X_test)

print("Naive Bayes Results:")
print(classification_report(y_test, y_pred_nb))
print("Accuracy:", accuracy_score(y_test, y_pred_nb))

Naive Bayes Results:
              precision    recall  f1-score   support

           0       0.72      0.77      0.75      2995
           1       0.73      0.67      0.70      2729

    accuracy                           0.72      5724
   macro avg       0.72      0.72      0.72      5724
weighted avg       0.72      0.72      0.72      5724

Accuracy: 0.7236198462613557


In [26]:
from sklearn.svm import SVC, LinearSVC

# Much faster for text data
svm = LinearSVC()
svm.fit(X_train, y_train)
y_pred_svm = svm.predict(X_test)

print("SVM Results:")
print(classification_report(y_test, y_pred_svm))
print("Accuracy:", accuracy_score(y_test, y_pred_svm))

SVM Results:
              precision    recall  f1-score   support

           0       0.73      0.77      0.75      2995
           1       0.73      0.68      0.71      2729

    accuracy                           0.73      5724
   macro avg       0.73      0.73      0.73      5724
weighted avg       0.73      0.73      0.73      5724

Accuracy: 0.730083857442348


In [28]:
from sklearn.neighbors import KNeighborsClassifier

# Try small K and fewer samples for speed
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train[:3000], y_train[:3000])  # smaller train set
y_pred_knn = knn.predict(X_test[:1000])  # smaller test set

print("KNN Results:")
print(classification_report(y_test[:1000], y_pred_knn))
print("Accuracy:", accuracy_score(y_test[:1000], y_pred_knn))

KNN Results:
              precision    recall  f1-score   support

           0       0.80      0.22      0.34       550
           1       0.49      0.93      0.65       450

    accuracy                           0.54      1000
   macro avg       0.65      0.58      0.49      1000
weighted avg       0.66      0.54      0.48      1000

Accuracy: 0.54


In [32]:
import pandas as pd

results = []

# Naive Bayes
nb_acc = accuracy_score(y_test, y_pred_nb)
results.append(["Naive Bayes", nb_acc])

# SVM
svm_acc = accuracy_score(y_test, y_pred_svm)
results.append(["Linear SVM", svm_acc])

# KNN (subset)
subset_size_test = 1000  # define again here
knn_acc = accuracy_score(y_test[:subset_size_test], y_pred_knn)
results.append(["KNN (subset)", knn_acc])

# Create DataFrame
results_df = pd.DataFrame(results, columns=["Model", "Accuracy"])
print(results_df)

          Model  Accuracy
0   Naive Bayes  0.723620
1    Linear SVM  0.730084
2  KNN (subset)  0.540000


In [34]:
import joblib

# Save model and vectorizer
joblib.dump(svm, "sarcasm_detector_model.pkl")
joblib.dump(vectorizer, "sarcasm_vectorizer.pkl")

print("✅ Model and vectorizer saved successfully!")


✅ Model and vectorizer saved successfully!


In [36]:
# Load model and vectorizer
loaded_model = joblib.load("sarcasm_detector_model.pkl")
loaded_vectorizer = joblib.load("sarcasm_vectorizer.pkl")


In [38]:
def predict_sarcasm(text):
    processed_text = preprocess(text)
    vec = loaded_vectorizer.transform([processed_text])
    prediction = loaded_model.predict(vec)[0]
    return "Sarcastic" if prediction == 1 else "Not Sarcastic"


In [46]:
print(predict_sarcasm("It was good to wait in sunlight for 3 hr"))
print(predict_sarcasm("Scientists discovered a cure for cancer"))


Sarcastic
Sarcastic
